### I221994 | Muhammad Qasim
### I222066 | Ayaan Khan
### I222018 | Ahmad Luqman

## Import Required Libraries

In [ ]:
import copy
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import trange
from collections import namedtuple

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
import torchvision
import torchvision.transforms as transforms

from sklearn.decomposition import PCA
from sklearn.svm import OneClassSVM
from sklearn.cluster import AgglomerativeClustering
from sklearn.neighbors import KNeighborsClassifier

# Configure plotting
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("Libraries imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

In [ ]:
def auror_defense(grad_vectors, attention_heads=4):
    """
    AUROR: Multi-head attention mechanism to identify malicious updates
    Uses self-attention to weigh client contributions
    
    Args:
        grad_vectors: List of gradient vectors from clients
        attention_heads: Number of attention heads
    
    Returns:
        benign_mask: Boolean array indicating benign clients
    """
    n_clients = len(grad_vectors)
    X = np.array(grad_vectors)
    
    # Simplified multi-head attention
    head_scores = []
    
    for h in range(attention_heads):
        # Random projection for each head
        np.random.seed(42 + h)
        proj_dim = min(100, X.shape[1] // 2)
        W = np.random.randn(X.shape[1], proj_dim) * 0.01
        
        # Project gradients
        X_proj = X @ W
        
        # Compute attention scores (similarity to mean)
        mean_proj = np.mean(X_proj, axis=0)
        scores = np.zeros(n_clients)
        
        for i in range(n_clients):
            # Cosine similarity to mean
            scores[i] = np.dot(X_proj[i], mean_proj) / (
                np.linalg.norm(X_proj[i]) * np.linalg.norm(mean_proj) + 1e-8
            )
        
        head_scores.append(scores)
    
    # Average scores across heads
    avg_scores = np.mean(head_scores, axis=0)
    
    # Select top clients (above median)
    threshold = np.median(avg_scores)
    benign_mask = avg_scores >= threshold
    
    return benign_mask

In [ ]:
def rfa_defense(grad_vectors, max_iter=100, tol=1e-5):
    """
    RFA: Uses geometric median instead of mean (more robust to outliers)
    Weierstrass algorithm for geometric median computation
    
    Args:
        grad_vectors: List of gradient vectors from clients
        max_iter: Maximum iterations for geometric median
        tol: Convergence tolerance
    
    Returns:
        benign_mask: Boolean array indicating benign clients
    """
    n_clients = len(grad_vectors)
    X = np.array(grad_vectors)
    
    # Compute geometric median using Weierstrass algorithm
    median = np.median(X, axis=0)  # Initialize with coordinate median
    
    for iteration in range(max_iter):
        prev_median = median.copy()
        
        # Compute distances to current median
        distances = np.linalg.norm(X - median, axis=1)
        
        # Avoid division by zero
        distances = np.maximum(distances, 1e-8)
        
        # Weighted update
        weights = 1.0 / distances
        weights = weights / np.sum(weights)
        
        median = np.sum(X * weights[:, np.newaxis], axis=0)
        
        # Check convergence
        if np.linalg.norm(median - prev_median) < tol:
            break
    
    # Compute distances to geometric median
    distances_to_median = np.linalg.norm(X - median, axis=1)
    
    # Use MAD (Median Absolute Deviation) for robust thresholding
    mad = np.median(np.abs(distances_to_median - np.median(distances_to_median)))
    threshold = np.median(distances_to_median) + 2.5 * mad
    
    benign_mask = distances_to_median <= threshold
    
    return benign_mask

## Model Definitions

In [ ]:
class SimpleCNN(nn.Module):
    """Simple CNN for MNIST and Fashion-MNIST"""
    def __init__(self, input_channels=1, num_classes=10):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(input_channels, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64*7*7 if input_channels==1 else 64*8*8, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )
    
    def forward(self, x):
        return self.fc(self.conv(x))

def get_resnet18(num_classes=10):
    """ResNet18 adapted for CIFAR-10"""
    model = torchvision.models.resnet18(pretrained=False)
    model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    model.maxpool = nn.Identity()
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model

print("Model architectures defined!")

## Utility Functions

In [ ]:
def model_to_vector(model):
    """Convert model parameters to a flat numpy vector"""
    return torch.cat([p.detach().flatten() for p in model.parameters()]).cpu().numpy()

def vector_to_model(model, vec):
    """Load vector into model parameters"""
    ptr = 0
    for p in model.parameters():
        num = p.numel()
        p.data.copy_(torch.tensor(vec[ptr:ptr+num]).view_as(p.data))
        ptr += num

def subtract_model_params(m_new, m_old):
    """Compute parameter update: new - old"""
    v_new = model_to_vector(m_new)
    v_old = model_to_vector(m_old)
    return v_new - v_old

def set_seed(seed):
    """Set random seeds for reproducibility"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

print("Utility functions defined!")

## Attack Implementations

In [ ]:
def sign_flip_update(grad_vec):
    """Sign flipping attack: negate the gradient update"""
    return -grad_vec

def lie_attack_simulation(global_mean, global_std, beta=1.5):
    """LIE attack: craft malicious update near mean but in negative direction"""
    sign = -1.0
    return global_mean + sign * beta * global_std

print("Attack functions defined!")

## Federated Learning Utilities

In [ ]:
def local_train(model, train_loader, device, epochs=1, lr=0.01):
    """Train model locally for specified epochs"""
    model.train()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr)
    
    for e in range(epochs):
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            out = model(x)
            loss = criterion(out, y)
            loss.backward()
            optimizer.step()

def fedavg(models):
    """Federated Averaging: average model parameters"""
    avg = {}
    for k in models[0].keys():
        avg[k] = sum([m[k].float() for m in models]) / len(models)
    return avg

def evaluate(model, dataloader, device):
    """Evaluate model accuracy on test set"""
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            preds = out.argmax(dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)
    
    return correct / total

print("Federated learning functions defined!")

## NC-FLD Defense: Weight Pruning, PCA, and Classification

In [ ]:
def select_top_by_magnitude(grad_vec, keep_ratio):
    """Select top parameters by magnitude (weight pruning step)"""
    magnitudes = np.abs(grad_vec)
    threshold = np.quantile(magnitudes, 1 - keep_ratio)
    mask = magnitudes > threshold
    return grad_vec[mask], mask

def build_feature_matrix(selected_vectors):
    """Build feature matrix from selected parameters (pad to max length)"""
    max_len = max([v.shape[0] for v in selected_vectors])
    X = np.zeros((len(selected_vectors), max_len), dtype=np.float32)
    for i, v in enumerate(selected_vectors):
        X[i, :v.shape[0]] = v
    return X

def apply_pca(X, n_components=2):
    """Apply PCA for dimensionality reduction"""
    pca = PCA(n_components=n_components)
    Z = pca.fit_transform(X)
    return Z, pca

def classify_embeddings(Z, method='ocsvm'):
    """Classify clients as benign (True) or malicious (False)"""
    if method == 'ocsvm':
        clf = OneClassSVM(kernel='rbf', nu=0.1, gamma='scale')
        clf.fit(Z)
        preds = clf.predict(Z)  # +1 inlier, -1 outlier
        return (preds == 1).astype(bool)
    
    elif method == 'agg':
        cl = AgglomerativeClustering(n_clusters=2, linkage='average')
        labels = cl.fit_predict(Z)
        counts = np.bincount(labels)
        benign_label = np.argmax(counts)
        return (labels == benign_label).astype(bool)
    
    elif method == 'knn':
        k = 4
        kn = KNeighborsClassifier(n_neighbors=k, metric='manhattan')
        agg = AgglomerativeClustering(n_clusters=2)
        pseudo = agg.fit_predict(Z)
        counts = np.bincount(pseudo)
        benign_label = np.argmax(counts)
        y = (pseudo == benign_label).astype(int)
        kn.fit(Z, y)
        preds = kn.predict(Z)
        return (preds == 1).astype(bool)
    elif method == 'auror':
        aur = auror_defense(Z, attention_heads=4)
        return aur
    elif method == 'rfa':
        rffaa = rfa_defense(Z, max_iter=100)
        return rffaa
        
    
    else:
        raise ValueError(f'Unknown defense method: {method}')

print("NC-FLD defense functions defined!")

## Main Simulation Function

In [ ]:
# Create named tuple for experiment configuration
ExpConfig = namedtuple('ExpConfig', ['dataset', 'clients', 'rounds', 'local_epochs', 
                                     'lr', 'attack', 'malicious_ratio', 'defense', 
                                     'keep_ratio', 'seed', 'log_interval'])

def simulate(config):
    """
    Run federated learning simulation with NC-FLD defense
    
    Args:
        config: ExpConfig object with experiment parameters
    
    Returns:
        results: List of test accuracies per round
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    set_seed(config.seed)
    
    # Dataset setup
    if config.dataset == 'mnist':
        transform = transforms.Compose([transforms.ToTensor(), 
                                       transforms.Normalize((0.5,), (0.5,))])
        trainset = torchvision.datasets.MNIST(root='./data', train=True, 
                                             download=True, transform=transform)
        testset = torchvision.datasets.MNIST(root='./data', train=False, 
                                            download=True, transform=transform)
        model_fn = lambda: SimpleCNN(input_channels=1, num_classes=10)
        batch_size = 32
        flip_map = {7: 1}
    
    elif config.dataset == 'fmnist':
        transform = transforms.Compose([transforms.ToTensor(), 
                                       transforms.Normalize((0.5,), (0.5,))])
        trainset = torchvision.datasets.FashionMNIST(root='./data', train=True, 
                                                     download=True, transform=transform)
        testset = torchvision.datasets.FashionMNIST(root='./data', train=False, 
                                                    download=True, transform=transform)
        model_fn = lambda: SimpleCNN(input_channels=1, num_classes=10)
        batch_size = 25
        flip_map = {0: 6, 6: 0}
    
    elif config.dataset == 'cifar10':
        transform = transforms.Compose([transforms.ToTensor(), 
                                       transforms.Normalize((0.5, 0.5, 0.5), 
                                                          (0.5, 0.5, 0.5))])
        trainset = torchvision.datasets.CIFAR10(root='./data', train=True, 
                                               download=True, transform=transform)
        testset = torchvision.datasets.CIFAR10(root='./data', train=False, 
                                              download=True, transform=transform)
        model_fn = lambda: get_resnet18(num_classes=10)
        batch_size = 100
        flip_map = {5: 3}  # dog -> cat
    
    else:
        raise NotImplementedError(f'Dataset {config.dataset} not implemented')
    
    # Create client data shards
    total_train = len(trainset)
    indices = list(range(total_train))
    random.shuffle(indices)
    shards = np.array_split(indices, config.clients)
    
    client_loaders = []
    for s in shards:
        sub = Subset(trainset, s)
        loader = DataLoader(sub, batch_size=batch_size, shuffle=True)
        client_loaders.append(loader)
    
    testloader = DataLoader(testset, batch_size=256, shuffle=False)
    
    # Select malicious clients
    num_mal = int(config.malicious_ratio * config.clients)
    mal_indices = set(random.sample(range(config.clients), num_mal))
    print(f"Malicious clients: {sorted(list(mal_indices))} (count={num_mal})")
    
    # Initialize global model
    global_model = model_fn().to(device)
    global_state = copy.deepcopy(global_model.state_dict())
    
    # Main federated learning rounds
    results = []
    for r in trange(config.rounds, desc='Training Rounds'):
        local_states = []
        grad_vectors = []
        raw_masks = []
        
        # Local training for each client
        for c in range(config.clients):
            model_c = model_fn().to(device)
            model_c.load_state_dict(global_state)
            old_model = copy.deepcopy(model_c)
            
            # Label flip attack: poison training data
            if config.attack == 'label_flip' and c in mal_indices:
                ds_indices = shards[c]
                poison_list = []
                for idx in ds_indices:
                    x, y = trainset[idx]
                    y_ = flip_map.get(int(y), int(y))
                    poison_list.append((x, y_))
                poisoned_loader = DataLoader(poison_list, batch_size=batch_size, shuffle=True)
                local_train(model_c, poisoned_loader, device, 
                          epochs=config.local_epochs, lr=config.lr)
            else:
                local_train(model_c, client_loaders[c], device, 
                          epochs=config.local_epochs, lr=config.lr)
            
            # Compute update vector
            diff_vec = subtract_model_params(model_c, old_model)
            
            # Model poisoning attacks
            if c in mal_indices and config.attack == 'sign_flip':
                diff_vec = sign_flip_update(diff_vec)
            
            if c in mal_indices and config.attack == 'lie':
                if len(grad_vectors) > 0:
                    global_mean = np.mean(np.stack(grad_vectors), axis=0)
                    global_std = np.std(np.stack(grad_vectors), axis=0) + 1e-6
                    diff_vec = lie_attack_simulation(global_mean, global_std, beta=1.0)
                else:
                    diff_vec = sign_flip_update(diff_vec)
            
            grad_vectors.append(diff_vec)
            local_states.append(model_c.state_dict())
        
        # NC-FLD Defense: Weight Pruning
        selected_vecs = []
        for vec in grad_vectors:
            sel, mask = select_top_by_magnitude(vec, config.keep_ratio)
            selected_vecs.append(sel)
            raw_masks.append(mask)
        
        # Build feature matrix and apply PCA
        X = build_feature_matrix(selected_vecs)
        Z, pca = apply_pca(X, n_components=2)
        
        # Classify clients
        benign_mask = classify_embeddings(Z, method=config.defense)
        benign_indices = [i for i, b in enumerate(benign_mask) if b]
        
        if len(benign_indices) == 0:
            print("Warning: all clients classified malicious; skipping aggregation")
            continue
        
        # Aggregate benign clients (FedAvg)
        benign_states = [local_states[i] for i in benign_indices]
        new_global_state = fedavg(benign_states)
        global_state = new_global_state
        
        # Evaluate
        global_model.load_state_dict(global_state)
        acc = evaluate(global_model, testloader, device)
        results.append(acc)
        
        if (r + 1) % config.log_interval == 0:
            print(f"Round {r+1}/{config.rounds} - Test Acc: {acc:.4f} - "
                  f"Included: {len(benign_indices)}/{config.clients}")
    
    final_acc = results[-1] if len(results) > 0 else None
    print(f"Final test accuracy: {final_acc:.4f}")
    
    # Save the final model
    import os
    os.makedirs('models', exist_ok=True)
    model_filename = f"models/{config.dataset}_{config.attack}_mr{config.malicious_ratio}_{config.defense}_final.pth"
    torch.save(global_model.state_dict(), model_filename)
    print(f"Model saved to: {model_filename}")
    
    return results, final_acc

print("Simulation function defined!")

## Initialize Results Storage

In [ ]:
# Storage for all experiment results
all_results = []

print("Results storage initialized!")

---
# MNIST Experiments (100 clients)
## Label Flip Attack

In [ ]:
# MNIST - Label Flip - Malicious Ratio 0.1 - OCSVM
config = ExpConfig(dataset='mnist', clients=100, rounds=100, local_epochs=1, lr=0.01,
                   attack='label_flip', malicious_ratio=0.1, defense='ocsvm',
                   keep_ratio=0.15, seed=42, log_interval=10)
round_accs, final_acc = simulate(config)
all_results.append({'dataset': 'mnist', 'attack': 'label_flip', 'malicious_ratio': 0.1,
                   'defense': 'ocsvm', 'final_accuracy': final_acc, 'round_accuracies': round_accs})

In [ ]:
# MNIST - Label Flip - Malicious Ratio 0.1 - AGG
config = ExpConfig(dataset='mnist', clients=100, rounds=100, local_epochs=1, lr=0.01,
                   attack='label_flip', malicious_ratio=0.1, defense='agg',
                   keep_ratio=0.15, seed=42, log_interval=10)
round_accs, final_acc = simulate(config)
all_results.append({'dataset': 'mnist', 'attack': 'label_flip', 'malicious_ratio': 0.1,
                   'defense': 'agg', 'final_accuracy': final_acc, 'round_accuracies': round_accs})

In [ ]:
# MNIST - Label Flip - Malicious Ratio 0.1 - KNN
config = ExpConfig(dataset='mnist', clients=100, rounds=100, local_epochs=1, lr=0.01,
                   attack='label_flip', malicious_ratio=0.1, defense='knn',
                   keep_ratio=0.15, seed=42, log_interval=10)
round_accs, final_acc = simulate(config)
all_results.append({'dataset': 'mnist', 'attack': 'label_flip', 'malicious_ratio': 0.1,
                   'defense': 'knn', 'final_accuracy': final_acc, 'round_accuracies': round_accs})

In [ ]:
# MNIST - Label Flip - Malicious Ratio 0.2 - All Defenses
for defense in ['ocsvm', 'agg', 'knn']:
    config = ExpConfig(dataset='mnist', clients=100, rounds=100, local_epochs=1, lr=0.01,
                       attack='label_flip', malicious_ratio=0.2, defense=defense,
                       keep_ratio=0.15, seed=42, log_interval=10)
    round_accs, final_acc = simulate(config)
    all_results.append({'dataset': 'mnist', 'attack': 'label_flip', 'malicious_ratio': 0.2,
                       'defense': defense, 'final_accuracy': final_acc, 'round_accuracies': round_accs})

In [ ]:
# MNIST - Label Flip - Malicious Ratio 0.3 - All Defenses
for defense in ['ocsvm', 'agg', 'knn']:
    config = ExpConfig(dataset='mnist', clients=100, rounds=100, local_epochs=1, lr=0.01,
                       attack='label_flip', malicious_ratio=0.3, defense=defense,
                       keep_ratio=0.15, seed=42, log_interval=10)
    round_accs, final_acc = simulate(config)
    all_results.append({'dataset': 'mnist', 'attack': 'label_flip', 'malicious_ratio': 0.3,
                       'defense': defense, 'final_accuracy': final_acc, 'round_accuracies': round_accs})

In [ ]:
# MNIST - Label Flip - Malicious Ratio 0.4 - All Defenses
for defense in ['ocsvm', 'agg', 'knn']:
    config = ExpConfig(dataset='mnist', clients=100, rounds=100, local_epochs=1, lr=0.01,
                       attack='label_flip', malicious_ratio=0.4, defense=defense,
                       keep_ratio=0.15, seed=42, log_interval=10)
    round_accs, final_acc = simulate(config)
    all_results.append({'dataset': 'mnist', 'attack': 'label_flip', 'malicious_ratio': 0.4,
                       'defense': defense, 'final_accuracy': final_acc, 'round_accuracies': round_accs})

## Sign Flip Attack

In [ ]:
# MNIST - Sign Flip - All Malicious Ratios and Defenses
for ratio in [0.1, 0.2, 0.3, 0.4]:
    for defense in ['ocsvm', 'agg', 'knn']:
        config = ExpConfig(dataset='mnist', clients=100, rounds=100, local_epochs=1, lr=0.01,
                           attack='sign_flip', malicious_ratio=ratio, defense=defense,
                           keep_ratio=0.15, seed=42, log_interval=10)
        round_accs, final_acc = simulate(config)
        all_results.append({'dataset': 'mnist', 'attack': 'sign_flip', 'malicious_ratio': ratio,
                           'defense': defense, 'final_accuracy': final_acc, 'round_accuracies': round_accs})

In [ ]:
# MNIST - Sign Flip - Malicious Ratio 0.3 - KNN
config = ExpConfig(dataset='mnist', clients=100, rounds=100, local_epochs=1, lr=0.01,
                   attack='sign_flip', malicious_ratio=0.3, defense='knn',
                   keep_ratio=0.15, seed=42, log_interval=10)
round_accs, final_acc = simulate(config)
all_results.append({'dataset': 'mnist', 'attack': 'sign_flip', 'malicious_ratio': 0.3,
                   'defense': 'knn', 'final_accuracy': final_acc, 'round_accuracies': round_accs})

In [ ]:
# MNIST - Sabel Flip - Malicious Ratio 0.3 - AGG
config = ExpConfig(dataset='mnist', clients=100, rounds=100, local_epochs=1, lr=0.01,
                   attack='sign_flip', malicious_ratio=0.3, defense='agg',
                   keep_ratio=0.15, seed=42, log_interval=10)
round_accs, final_acc = simulate(config)
all_results.append({'dataset': 'mnist', 'attack': 'sign_flip', 'malicious_ratio': 0.3,
                   'defense': 'agg', 'final_accuracy': final_acc, 'round_accuracies': round_accs})

In [ ]:
# MNIST - Sabel Flip - Malicious Ratio 0.4 - ocsvm
config = ExpConfig(dataset='mnist', clients=100, rounds=100, local_epochs=1, lr=0.01,
                   attack='sign_flip', malicious_ratio=0.4, defense='ocsvm',
                   keep_ratio=0.15, seed=42, log_interval=10)
round_accs, final_acc = simulate(config)
all_results.append({'dataset': 'mnist', 'attack': 'sign_flip', 'malicious_ratio': 0.4,
                   'defense': 'ocsvm', 'final_accuracy': final_acc, 'round_accuracies': round_accs})

In [ ]:
# MNIST - Sabel Flip - Malicious Ratio 0.4 - agg
config = ExpConfig(dataset='mnist', clients=100, rounds=100, local_epochs=1, lr=0.01,
                   attack='sign_flip', malicious_ratio=0.4, defense='agg',
                   keep_ratio=0.15, seed=42, log_interval=10)
round_accs, final_acc = simulate(config)
all_results.append({'dataset': 'mnist', 'attack': 'sign_flip', 'malicious_ratio': 0.4,
                   'defense': 'agg', 'final_accuracy': final_acc, 'round_accuracies': round_accs})

In [ ]:
# MNIST - Sabel Flip - Malicious Ratio 0.4 - knn
config = ExpConfig(dataset='mnist', clients=100, rounds=100, local_epochs=1, lr=0.01,
                   attack='sign_flip', malicious_ratio=0.4, defense='knn',
                   keep_ratio=0.15, seed=42, log_interval=10)
round_accs, final_acc = simulate(config)
all_results.append({'dataset': 'mnist', 'attack': 'sign_flip', 'malicious_ratio': 0.4,
                   'defense': 'knn', 'final_accuracy': final_acc, 'round_accuracies': round_accs})

## LIE Attack

In [ ]:
# MNIST - LIE - All Defenses
config = ExpConfig(dataset='mnist', clients=100, rounds=100, local_epochs=1, lr=0.01,
                   attack='lie', malicious_ratio=0.1, defense='ocsvm',
                   keep_ratio=0.15, seed=42, log_interval=10)
round_accs, final_acc = simulate(config)
all_results.append({'dataset': 'mnist', 'attack': 'lie', 'malicious_ratio': 0.1,
                   'defense': 'ocsvm', 'final_accuracy': final_acc, 'round_accuracies': round_accs})

In [ ]:
config = ExpConfig(dataset='mnist', clients=100, rounds=100, local_epochs=1, lr=0.01,
                       attack='lie', malicious_ratio=0.1, defense='agg',
                       keep_ratio=0.15, seed=42, log_interval=10)
round_accs, final_acc = simulate(config)
all_results.append({'dataset': 'mnist', 'attack': 'lie', 'malicious_ratio': 0.1,
                       'defense': 'agg', 'final_accuracy': final_acc, 'round_accuracies': round_accs})

In [ ]:
config = ExpConfig(dataset='mnist', clients=100, rounds=100, local_epochs=1, lr=0.01,
                       attack='lie', malicious_ratio=0.1, defense='knn',
                       keep_ratio=0.15, seed=42, log_interval=10)
round_accs, final_acc = simulate(config)
all_results.append({'dataset': 'mnist', 'attack': 'lie', 'malicious_ratio': 0.1,
                   'defense': 'knn', 'final_accuracy': final_acc, 'round_accuracies': round_accs})

In [ ]:
config = ExpConfig(dataset='mnist', clients=100, rounds=100, local_epochs=1, lr=0.01,
                   attack='lie', malicious_ratio=0.2, defense='ocsvm',
                   keep_ratio=0.15, seed=42, log_interval=10)
round_accs, final_acc = simulate(config)
all_results.append({'dataset': 'mnist', 'attack': 'lie', 'malicious_ratio': 0.2,
                   'defense': 'ocsvm', 'final_accuracy': final_acc, 'round_accuracies': round_accs})

In [ ]:
config = ExpConfig(dataset='mnist', clients=100, rounds=100, local_epochs=1, lr=0.01,
                           attack='lie', malicious_ratio=0.2, defense='agg',
                           keep_ratio=0.15, seed=42, log_interval=10)
round_accs, final_acc = simulate(config)
all_results.append({'dataset': 'mnist', 'attack': 'lie', 'malicious_ratio': 0.2,
                           'defense': 'agg', 'final_accuracy': final_acc, 'round_accuracies': round_accs})

In [ ]:
config = ExpConfig(dataset='mnist', clients=100, rounds=100, local_epochs=1, lr=0.01,
                           attack='lie', malicious_ratio=0.2, defense='knn',
                           keep_ratio=0.15, seed=42, log_interval=10)
round_accs, final_acc = simulate(config)
all_results.append({'dataset': 'mnist', 'attack': 'lie', 'malicious_ratio': 0.2,
                           'defense': 'knn', 'final_accuracy': final_acc, 'round_accuracies': round_accs})

In [ ]:
for defense in ['ocsvm', 'agg', 'knn']:
        config = ExpConfig(dataset='mnist', clients=100, rounds=100, local_epochs=1, lr=0.01,
                           attack='lie', malicious_ratio=0.3, defense=defense,
                           keep_ratio=0.15, seed=42, log_interval=10)
        round_accs, final_acc = simulate(config)
        all_results.append({'dataset': 'mnist', 'attack': 'lie', 'malicious_ratio': 0.3,
                           'defense': defense, 'final_accuracy': final_acc, 'round_accuracies': round_accs})

In [ ]:
for defense in ['ocsvm', 'agg', 'knn']:
        config = ExpConfig(dataset='mnist', clients=100, rounds=100, local_epochs=1, lr=0.01,
                           attack='lie', malicious_ratio=0.4, defense=defense,
                           keep_ratio=0.15, seed=42, log_interval=10)
        round_accs, final_acc = simulate(config)
        all_results.append({'dataset': 'mnist', 'attack': 'lie', 'malicious_ratio': 0.4,
                           'defense': defense, 'final_accuracy': final_acc, 'round_accuracies': round_accs})

In [ ]:
for defense in ['agg', 'knn']:
        config = ExpConfig(dataset='mnist', clients=100, rounds=100, local_epochs=1, lr=0.01,
                           attack='lie', malicious_ratio=0.4, defense=defense,
                           keep_ratio=0.15, seed=42, log_interval=10)
        round_accs, final_acc = simulate(config)
        all_results.append({'dataset': 'mnist', 'attack': 'lie', 'malicious_ratio': 0.4,
                           'defense': defense, 'final_accuracy': final_acc, 'round_accuracies': round_accs})

---
# Results Analysis and Visualization

## Load All Saved Models

In [ ]:
import os
import glob

# Dictionary to store all loaded models
loaded_models = {}

# Get all model files
model_files = glob.glob('models/*.pth')

print(f"Found {len(model_files)} saved models\n")
print("Loading models...\n")

for model_path in sorted(model_files):
    # Extract configuration from filename
    filename = os.path.basename(model_path)
    parts = filename.replace('_final.pth', '').split('_')
    
    # Parse filename: dataset_attack_mrX.X_defense_final.pth
    if len(parts) >= 4:
        dataset = parts[0]
        # Handle multi-word attacks like 'sign_flip'
        if parts[1] == 'sign' or parts[1] == 'label':
            attack = f"{parts[1]}_{parts[2]}"
            malicious_ratio = parts[3].replace('mr', '')
            defense = parts[4] if len(parts) > 4 else 'unknown'
        else:
            attack = parts[1]
            malicious_ratio = parts[2].replace('mr', '')
            defense = parts[3] if len(parts) > 3 else 'unknown'
        
        # Determine model architecture
        if dataset == 'cifar10':
            model = get_resnet18(num_classes=10)
        else:  # mnist or fmnist
            model = SimpleCNN(input_channels=1, num_classes=10)
        
        # Load model weights
        try:
            model.load_state_dict(torch.load(model_path, map_location='cpu'))
            model.eval()
            
            # Store with descriptive key
            key = f"{dataset}_{attack}_mr{malicious_ratio}_{defense}"
            loaded_models[key] = {
                'model': model,
                'dataset': dataset,
                'attack': attack,
                'malicious_ratio': float(malicious_ratio),
                'defense': defense,
                'path': model_path
            }
            
            print(f"✓ Loaded: {key}")
        except Exception as e:
            print(f"✗ Failed to load {filename}: {str(e)}")

print(f"\n{'='*80}")
print(f"Successfully loaded {len(loaded_models)} models")
print(f"{'='*80}")

# Display summary
print("\nLoaded models summary:")
for key in sorted(loaded_models.keys()):
    info = loaded_models[key]
    print(f"  {key}: {info['dataset'].upper()} | {info['attack']} | MR={info['malicious_ratio']} | {info['defense'].upper()}")

In [ ]:
# Reconstruct results from loaded models by evaluating them
print("Reconstructing experiment results from loaded models...")
print("This will evaluate each model on its test dataset.\n")

# Check if we have loaded models
if len(loaded_models) == 0:
    print(" No models loaded. Please run the model loading cell first.")
else:
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    reconstructed_results = []
    
    # Prepare test datasets (cached to avoid repeated downloads)
    print("Loading test datasets...")
    
    # MNIST
    transform_mnist = transforms.Compose([transforms.ToTensor(), 
                                          transforms.Normalize((0.5,), (0.5,))])
    testset_mnist = torchvision.datasets.MNIST(root='./data', train=False, 
                                               download=True, transform=transform_mnist)
    testloader_mnist = DataLoader(testset_mnist, batch_size=256, shuffle=False)
    
    
    testloaders = {
        'mnist': testloader_mnist
    }
    
    print(f"\nEvaluating {len(loaded_models)} models...\n")
    
    # Evaluate each loaded model
    for key, model_info in loaded_models.items():
        model = model_info['model'].to(device)
        dataset = model_info['dataset']
        attack = model_info['attack']
        malicious_ratio = model_info['malicious_ratio']
        defense = model_info['defense']
        
        # Get appropriate test loader
        testloader = testloaders[dataset]
        
        # Evaluate model
        print(f"Evaluating: {key}...", end=' ')
        accuracy = evaluate(model, testloader, device)
        print(f"Accuracy: {accuracy:.4f}")
        
        # Add to reconstructed results
        reconstructed_results.append({
            'dataset': dataset,
            'attack': attack,
            'malicious_ratio': malicious_ratio,
            'defense': defense,
            'final_accuracy': accuracy
        })
    
    # Create DataFrame from reconstructed results
    df_reconstructed = pd.DataFrame(reconstructed_results)
    
    # Save to CSV
    df_reconstructed.to_csv('nc_fld_experiment_results.csv', index=False)
    
    print(f"\n{'='*80}")
    print(f" Successfully reconstructed {len(df_reconstructed)} experiment results!")
    print(f" Saved to: nc_fld_experiment_results.csv")
    print(f"{'='*80}\n")
    
    # Display summary
    print("Reconstructed Results Summary:")
    print(f"Total experiments: {len(df_reconstructed)}")
    print(f"Datasets: {', '.join(df_reconstructed['dataset'].unique())}")
    print(f"Attacks: {', '.join(df_reconstructed['attack'].unique())}")
    print(f"Defenses: {', '.join(df_reconstructed['defense'].unique())}")
    print(f"Malicious ratios: {', '.join(map(str, sorted(df_reconstructed['malicious_ratio'].unique())))}")
    
    print(f"\n{'='*80}")
    print("Accuracy Statistics:")
    print(f"{'='*80}")
    print(df_reconstructed['final_accuracy'].describe())
    
    print("\n✓ You can now run the next cells to display full analysis and visualizations!")

In [ ]:
# Load results from CSV if available, otherwise use all_results
import os

if os.path.exists('nc_fld_experiment_results.csv') and len(all_results) == 0:
    print("Loading results from saved CSV file...")
    try:
        df = pd.read_csv('nc_fld_experiment_results.csv')
        print(f"✓ Loaded {len(df)} experiments from CSV\n")
    except pd.errors.EmptyDataError:
        print("⚠ CSV file is empty. Creating empty DataFrame...")
        df = pd.DataFrame(columns=['dataset', 'attack', 'malicious_ratio', 'defense', 'final_accuracy'])
elif len(all_results) > 0:
    print("Creating DataFrame from experiment results...")
    df = pd.DataFrame([{k: v for k, v in r.items() if k != 'round_accuracies'} for r in all_results])
    # Save to CSV
    df.to_csv('nc_fld_experiment_results.csv', index=False)
    print(f"✓ Created and saved {len(df)} experiments to CSV\n")
else:
    print("⚠ No results found. Please run experiments first or ensure CSV file exists.")
    print("Creating empty DataFrame with expected columns...")
    df = pd.DataFrame(columns=['dataset', 'attack', 'malicious_ratio', 'defense', 'final_accuracy'])

# Only display summary if we have data
if len(df) > 0:
    print(f"{'='*80}")
    print(f"EXPERIMENT RESULTS SUMMARY")
    print(f"{'='*80}")
    print(f"\nTotal experiments: {len(df)}")
    
    # Check if required columns exist
    if 'dataset' in df.columns:
        print(f"Datasets: {', '.join(df['dataset'].unique())}")
    if 'attack' in df.columns:
        print(f"Attacks: {', '.join(df['attack'].unique())}")
    if 'defense' in df.columns:
        print(f"Defenses: {', '.join(df['defense'].unique())}")
    if 'malicious_ratio' in df.columns:
        print(f"Malicious ratios: {', '.join(map(str, sorted(df['malicious_ratio'].unique())))}")
    
    # Display grouped results if columns exist
    if all(col in df.columns for col in ['dataset', 'attack', 'defense', 'final_accuracy']):
        print(f"\n{'='*80}")
        print("Results by Dataset, Attack, and Defense:")
        print(f"{'='*80}")
        summary = df.groupby(['dataset', 'attack', 'defense'])['final_accuracy'].agg(['mean', 'std', 'count'])
        print(summary.to_string())
        
        print(f"\n{'='*80}")
        print("Overall Statistics:")
        print(f"{'='*80}")
        print(df['final_accuracy'].describe())
    else:
        print("\n⚠ Some expected columns are missing from the data.")
else:
    print("\n⚠ No experiment data available to display.")

## Summary Tables

In [ ]:
# Create pivot table for easy viewing
summary_pivot = df.pivot_table(
    values='final_accuracy',
    index=['dataset', 'attack', 'malicious_ratio'],
    columns='defense',
    aggfunc='mean'
)

print("\n" + "="*80)
print("SUMMARY: Final Test Accuracy by Configuration")
print("="*80)
print(summary_pivot.to_string())

summary_pivot.to_csv('summary_accuracy_table.csv')
summary_pivot

In [ ]:
# Individual dataset tables
for dataset in df['dataset'].unique():
    print(f"\n{'='*80}")
    print(f"{dataset.upper()} - Detailed Results")
    print(f"{'='*80}")
    
    df_dataset = df[df['dataset'] == dataset]
    pivot = df_dataset.pivot_table(
        values='final_accuracy',
        index=['attack', 'malicious_ratio'],
        columns='defense',
        aggfunc='mean'
    )
    print(pivot.to_string())
    pivot.to_csv(f'{dataset}_results_table.csv')
    display(pivot)

## Visualization 1: Accuracy vs Malicious Ratio

In [ ]:
# Plot accuracy vs malicious ratio (MNIST only)
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('NC-FLD Defense Performance on MNIST: Accuracy vs Malicious Ratio', 
             fontsize=16, fontweight='bold')

# Filter for MNIST only
df_mnist = df[df['dataset'] == 'mnist']
attacks = df_mnist['attack'].unique()

for j, attack in enumerate(attacks):
    ax = axes[j]
    data = df_mnist[df_mnist['attack'] == attack]
    
    if len(data) > 0:
        for defense in ['ocsvm', 'agg', 'knn']:
            defense_data = data[data['defense'] == defense].sort_values('malicious_ratio')
            if len(defense_data) > 0:
                ax.plot(defense_data['malicious_ratio'], 
                       defense_data['final_accuracy'],
                       marker='o', label=defense.upper(), linewidth=2.5, markersize=8)
        
        ax.set_xlabel('Malicious Client Ratio', fontsize=12)
        ax.set_ylabel('Test Accuracy', fontsize=12)
        ax.set_title(f'{attack.replace("_", " ").title()} Attack', 
                    fontsize=13, fontweight='bold')
        ax.legend(fontsize=11)
        ax.grid(True, alpha=0.3)
        ax.set_ylim([0, 1])

plt.tight_layout()
plt.savefig('mnist_accuracy_vs_malicious_ratio.png', dpi=300, bbox_inches='tight')
plt.show()

## Visualization 2: Defense Method Comparison

In [ ]:
# Bar plot comparing defense methods (MNIST only)
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
fig.suptitle('Average Defense Performance on MNIST', fontsize=16, fontweight='bold')

# Filter for MNIST only
df_mnist = df[df['dataset'] == 'mnist']
defense_avg = df_mnist.groupby('defense')['final_accuracy'].mean().sort_values(ascending=False)

colors = ['#2ecc71', '#3498db', '#e74c3c']
bars = ax.bar(range(len(defense_avg)), defense_avg.values, color=colors, alpha=0.8, edgecolor='black', linewidth=2)
ax.set_xticks(range(len(defense_avg)))
ax.set_xticklabels([d.upper() for d in defense_avg.index], fontsize=14, fontweight='bold')
ax.set_ylabel('Average Test Accuracy', fontsize=13)
ax.set_title('Defense Method Comparison', fontsize=14, fontweight='bold', pad=20)
ax.set_ylim([0, 1])
ax.grid(axis='y', alpha=0.3)

for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height + 0.02,
           f'{height:.3f}', ha='center', va='bottom', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.savefig('mnist_defense_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

## Visualization 3: Attack Impact Heatmap

In [ ]:
# Heatmap showing attack impact (MNIST only)
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
fig.suptitle('Attack Impact on MNIST Accuracy (Averaged Across Defenses)', 
             fontsize=16, fontweight='bold')

# Filter for MNIST only
df_mnist = df[df['dataset'] == 'mnist']

heatmap_data = df_mnist.pivot_table(
    values='final_accuracy',
    index='attack',
    columns='malicious_ratio',
    aggfunc='mean'
)

sns.heatmap(heatmap_data, annot=True, fmt='.3f', cmap='RdYlGn', 
            vmin=0, vmax=1, ax=ax, cbar_kws={'label': 'Accuracy'},
            linewidths=1, linecolor='gray', annot_kws={'fontsize': 11})
ax.set_title('Attack Impact by Malicious Ratio', fontsize=14, fontweight='bold', pad=20)
ax.set_xlabel('Malicious Ratio', fontsize=13)
ax.set_ylabel('Attack Type', fontsize=13)
ax.set_yticklabels([a.replace('_', ' ').title() for a in heatmap_data.index], rotation=0, fontsize=12)
ax.set_xticklabels(ax.get_xticklabels(), fontsize=12)

plt.tight_layout()
plt.savefig('mnist_attack_impact_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

## Statistical Analysis

In [ ]:
print("\n" + "="*80)
print("STATISTICAL SUMMARY")
print("="*80)

print("\n1. Overall Accuracy Statistics:")
print(df['final_accuracy'].describe())

print("\n2. Best Performing Configurations (Top 10):")
top_10 = df.nlargest(10, 'final_accuracy')[['dataset', 'attack', 'malicious_ratio', 'defense', 'final_accuracy']]
print(top_10.to_string(index=False))

print("\n3. Worst Performing Configurations (Bottom 10):")
bottom_10 = df.nsmallest(10, 'final_accuracy')[['dataset', 'attack', 'malicious_ratio', 'defense', 'final_accuracy']]
print(bottom_10.to_string(index=False))

print("\n4. Average Accuracy by Defense Method:")
defense_stats = df.groupby('defense')['final_accuracy'].agg(['mean', 'std', 'min', 'max', 'count'])
print(defense_stats.to_string())

print("\n5. Average Accuracy by Attack Type:")
attack_stats = df.groupby('attack')['final_accuracy'].agg(['mean', 'std', 'min', 'max', 'count'])
print(attack_stats.to_string())

print("\n6. Average Accuracy by Dataset:")
dataset_stats = df.groupby('dataset')['final_accuracy'].agg(['mean', 'std', 'min', 'max', 'count'])
print(dataset_stats.to_string())

print("\n7. Average Accuracy by Malicious Ratio:")
ratio_stats = df.groupby('malicious_ratio')['final_accuracy'].agg(['mean', 'std', 'min', 'max'])
print(ratio_stats.to_string())

# Advanced New Defense

In [17]:
for ratio in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]:
    config = ExpConfig(dataset='mnist', clients=100, rounds=100, local_epochs=1, lr=0.01,
                       attack='label_flip', malicious_ratio=ratio, defense='rfa',
                       keep_ratio=0.15, seed=42, log_interval=10)
    round_accs, final_acc = simulate(config)
    all_results.append({'dataset': 'mnist', 'attack': 'label_flip', 'malicious_ratio': ratio,
                       'defense': 'rfa', 'final_accuracy': final_acc, 'round_accuracies': round_accs})

Malicious clients: [5, 8, 20, 41, 54, 59, 72, 80, 83, 84] (count=10)


Training Rounds:  10%|█         | 10/100 [02:35<23:11, 15.46s/it]

Round 10/100 - Test Acc: 0.6838 - Included: 88/100


Training Rounds:  20%|██        | 20/100 [05:11<21:07, 15.85s/it]

Round 20/100 - Test Acc: 0.8708 - Included: 81/100


...

Round 80/100 - Test Acc: 0.8529 - Included: 60/100


Training Rounds:  90%|█████████ | 90/100 [23:35<02:37, 15.76s/it]

Round 90/100 - Test Acc: 0.8568 - Included: 60/100


Training Rounds: 100%|██████████| 100/100 [26:12<00:00, 15.72s/it]

Round 100/100 - Test Acc: 0.8616 - Included: 60/100
Final test accuracy: 0.8616
Model saved to: models/mnist_label_flip_mr0.6_rfa_final.pth


In [18]:
for ratio in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]:
    config = ExpConfig(dataset='mnist', clients=100, rounds=100, local_epochs=1, lr=0.01,
                       attack='sign_flip', malicious_ratio=ratio, defense='rfa',
                       keep_ratio=0.15, seed=42, log_interval=10)
    round_accs, final_acc = simulate(config)
    all_results.append({'dataset': 'mnist', 'attack': 'sign_flip', 'malicious_ratio': ratio,
                       'defense': 'rfa', 'final_accuracy': final_acc, 'round_accuracies': round_accs})

Malicious clients: [5, 8, 20, 41, 54, 59, 72, 80, 83, 84] (count=10)


Training Rounds:  10%|█         | 10/100 [02:36<23:19, 15.55s/it]

Round 10/100 - Test Acc: 0.6845 - Included: 89/100


Training Rounds:  20%|██        | 20/100 [05:12<20:50, 15.63s/it]

Round 20/100 - Test Acc: 0.8705 - Included: 84/100


...

Training Rounds:  90%|█████████ | 90/100 [23:32<02:35, 15.56s/it]

Round 90/100 - Test Acc: 0.9554 - Included: 94/100


Training Rounds: 100%|██████████| 100/100 [26:09<00:00, 15.70s/it]

Round 100/100 - Test Acc: 0.9596 - Included: 90/100
Final test accuracy: 0.9596


Model saved to: models/mnist_sign_flip_mr0.6_rfa_final.pth


In [19]:
for ratio in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]:
    config = ExpConfig(dataset='mnist', clients=100, rounds=100, local_epochs=1, lr=0.01,
                       attack='lie', malicious_ratio=ratio, defense='rfa',
                       keep_ratio=0.15, seed=42, log_interval=10)
    round_accs, final_acc = simulate(config)
    all_results.append({'dataset': 'mnist', 'attack': 'lie', 'malicious_ratio': ratio,
                       'defense': 'rfa', 'final_accuracy': final_acc, 'round_accuracies': round_accs})

Malicious clients: [5, 8, 20, 41, 54, 59, 72, 80, 83, 84] (count=10)


Training Rounds:  10%|█         | 10/100 [02:40<24:04, 16.05s/it]

Round 10/100 - Test Acc: 0.6830 - Included: 91/100


Training Rounds:  20%|██        | 20/100 [05:21<21:30, 16.13s/it]

Round 20/100 - Test Acc: 0.8701 - Included: 81/100


...

Round 80/100 - Test Acc: 0.9506 - Included: 100/100


Training Rounds:  90%|█████████ | 90/100 [29:57<03:18, 19.86s/it]

Round 90/100 - Test Acc: 0.9562 - Included: 100/100


Training Rounds: 100%|██████████| 100/100 [33:18<00:00, 19.98s/it]

Round 100/100 - Test Acc: 0.9599 - Included: 100/100
Final test accuracy: 0.9599
Model saved to: models/mnist_lie_mr0.6_rfa_final.pth
